<a href="https://colab.research.google.com/github/sumitkuml/Garbage-classification/blob/master/Garbage-classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Garbage classification

In [0]:
 ! pip install -q kaggle

In [0]:
from google.colab import files

In [0]:
files.upload()

In [0]:
 ! mkdir ~/.kaggle 

In [0]:
! cp kaggle.json ~/.kaggle/

In [0]:
 ! chmod 600 ~/.kaggle/kaggle.json

In [0]:
 ! kaggle datasets list

In [0]:
!kaggle datasets download -d asdasdasasdas/garbage-classification

In [0]:
! mkdir garbage_classification

In [0]:
! unzip garbage-classification.zip -d garbage_classification

In [0]:
import numpy
import os
import cv2

In [0]:
x=[]
z=[]
cardboard_directory='/content/garbage_classification/Garbage classification/Garbage classification/cardboard/'
glass_directory='/content/garbage_classification/Garbage classification/Garbage classification/glass'
metal_directory='/content/garbage_classification/Garbage classification/Garbage classification/metal'
paper_directory='/content/garbage_classification/Garbage classification/Garbage classification/paper'
plastic_directory='/content/garbage_classification/Garbage classification/Garbage classification/plastic'
trash_directory='/content/garbage_classification/Garbage classification/Garbage classification/trash'

In [0]:

from tqdm import tqdm_notebook as tqdm
import numpy as np
IMG_SIZE=150
def assign_label(img,garbage_type):
  return garbage_type

In [0]:
def make_train_data(garbage_type,DIR):
  print(os.listdir(DIR))
  for img in tqdm(os.listdir(DIR)):
    label=assign_label(img,garbage_type)
    path=os.path.join(DIR,img)
    img=cv2.imread(path,cv2.IMREAD_COLOR)
    img=cv2.resize(img,(IMG_SIZE,IMG_SIZE))
    x.append(np.array(img))
    z.append(str(label))

In [0]:
make_train_data('cardboard',cardboard_directory)

In [0]:
make_train_data('glass',glass_directory)

In [0]:
make_train_data('metal',metal_directory)

In [0]:
make_train_data('paper',paper_directory)

In [0]:
make_train_data('plastic',plastic_directory)

In [0]:
make_train_data('trash',trash_directory)

In [0]:
x

In [0]:
import matplotlib.pyplot as plt

In [0]:
import random as rn
fig,ax=plt.subplots(5,2)
fig.set_size_inches(15,15)
for i in range(5):
  for j in range(2):
    l=rn.randint(0,len(z))
    ax[i,j].imshow(x[l])
    ax[i,j].set_title('garbage: '+ z[l])

In [0]:
#Label encoding for categorical variable
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score

In [0]:
le=LabelEncoder()

In [0]:
y=le.fit_transform(z)
y=to_categorical(y,6)
x=np.array(x)
x=x/255

In [0]:
print(len(x))
from keras.preprocessing.image import ImageDataGenerator

In [0]:

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [0]:
np.random.seed(42)
rn.seed(42)


In [0]:
#modelling the model
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D, Flatten, Dense

In [0]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
 

model.add(Conv2D(filters =96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters = 96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dense(6, activation = "softmax"))


In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [0]:
model.summary()

In [0]:
batch_size=128
epochs=25

In [0]:
from keras.preprocessing.image import ImageDataGenerator

In [0]:
datagen=ImageDataGenerator(shear_range=0.2,zoom_range=0.2,horizontal_flip=True,vertical_flip=True)

In [0]:
datagen.fit(x_train)

In [0]:
train_generator=datagen.flow(x_train,y_train,batch_size=batch_size)

In [0]:
history=model.fit_generator(train_generator,epochs=epochs,validation_data=(x_test,y_test),verbose=1,steps_per_epoch=x_train.shape[0]/batch_size)

In [0]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Modle Loss')
plt.xlabel('loss')
plt.ylabel('epochs')
plt.legend(['train','test'])
plt.show()

In [0]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Modle Loss')
plt.xlabel('accuracy')
plt.ylabel('epochs')
plt.legend(['train','test'])
plt.show()

In [0]:
#Test the image
from keras.preprocessing import image

In [0]:
uploaded=files.upload()
for fn in uploaded.keys():
  path=fn
  img =image.load_img(path,target_size=(150,150))
  x=image.img_to_array(img)
  x=np.expand_dims(x,axis=0)
  images=np.vstack([x])
  classes=model.predict(images,batch_size=5)
  classes_digits=np.argmax(classes,axis=1)
  print(fn)
  print(le.inverse_transform(classes_digits))

In [0]:
pred=model.predict(x_test)
pred_digits=np.argmax(pred,axis=1)
print(pred_digits)

In [0]:
# now storing some properly as well as misclassified indexes'.
i=0
prop_class=[]
mis_class=[]

for i in range(len(y_test)):
    if(np.argmax(y_test[i])==pred_digits[i]):
        prop_class.append(i)
    if(len(prop_class)==8):
        break

i=0
for i in range(len(y_test)):
    if(not np.argmax(y_test[i])==pred_digits[i]):
        mis_class.append(i)
    if(len(mis_class)==8):
        break

In [0]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')


count=0
fig,ax=plt.subplots(4,2)
fig.set_size_inches(15,15)
for i in range (4):
    for j in range (2):
        ax[i,j].imshow(x_test[prop_class[count]])
        ax[i,j].set_title("Predicted garbage : "+str(le.inverse_transform([pred_digits[prop_class[count]]]))+"\n"+"Actual garbage : "+str(le.inverse_transform(np.argmax([y_test[prop_class[count]]]))))
        plt.tight_layout()
        count+=1